In [1]:
import sys
sys.path.insert(0, '../')

from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

# Setup Mount Controller

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [13]:
start_time = time.time
n = 10
interval = 0.528 
for _ in range(n):
    KMC.rotate_simulataneously(0, 0, wait_for_elapsed_time=interval)
    KMC.rotate_simulataneously(25,25, wait_for_elapsed_time=interval)

print((time.time() - start_time) / (2 * n))


0.5343457341194153


In [19]:
import statistics

def adaptive_rotation(KMC, initial_interval, target_interval, n):
    """
    Rotates the motors adaptively to maintain the desired interval.
    
    Parameters:
        KMC: The motor control object.
        initial_interval: Initial interval between rotations (in seconds).
        target_interval: Desired interval between rotations (in seconds).
        n: Number of rotations to perform.
    """
    interval = initial_interval
    total_rotations = 2 * n  # Two rotations per cycle (0->0 and 25->25)
    time_log = []  # Log actual timings for analysis

    for _ in range(n):
        # Rotate to angle 0
        start_time = time.time()
        KMC.rotate_simulataneously(0, 0, wait_for_elapsed_time=interval)
        elapsed = time.time() - start_time
        time_log.append(elapsed)

        # Adjust interval based on measured time
        interval += target_interval - elapsed

        # Rotate to angle 25
        start_time = time.time()
        KMC.rotate_simulataneously(25, 25, wait_for_elapsed_time=interval)
        elapsed = time.time() - start_time
        time_log.append(elapsed)

        # Adjust interval again
        interval += target_interval - elapsed

    # Calculate and print statistics
    mean_time = statistics.mean(time_log)
    std_dev_time = statistics.stdev(time_log)

    print(f"Mean interval: {mean_time:.6f}s")
    print(f"Standard deviation: {std_dev_time:.6f}s")
    print(f"Time log (s): {time_log}")

# Example usage
initial_interval = 0.536  # Start with the desired interval
target_interval = 0.536   # Desired interval
n = 10                    # Number of cycles

adaptive_rotation(KMC, initial_interval, target_interval, n)



Mean interval: 0.536344s
Standard deviation: 0.007754s
Time log (s): [0.5503997802734375, 0.5345752239227295, 0.5308890342712402, 0.5425076484680176, 0.5281710624694824, 0.5306782722473145, 0.5442988872528076, 0.5269861221313477, 0.5484797954559326, 0.5294992923736572, 0.5448257923126221, 0.5295403003692627, 0.543318510055542, 0.5283842086791992, 0.5310087203979492, 0.5454626083374023, 0.5324938297271729, 0.531613826751709, 0.5318515300750732, 0.5418930053710938]


In [21]:
import time
import statistics
import random

def adaptive_rotation_efficient(KMC, initial_interval, target_interval, n):
    """
    Rotates motors adaptively to maintain the desired interval with randomized rotation patterns.
    
    Parameters:
        KMC: The motor control object.
        initial_interval: Initial interval between rotations (in seconds).
        target_interval: Desired interval between rotations (in seconds).
        n: Number of clicks to perform.
    """
    interval = initial_interval
    time_log = []  # Log actual timings for analysis

    for _ in range(n):
        # Determine rotation pattern: 0 (Alice), 1 (Bob), 2 (Both)
        rotation_choice = random.choice([0, 1, 2])

        # Set angles based on the choice
        if rotation_choice == 0:  # Only Alice rotates
            alice_angle, bob_angle = 0, KMC.bob.get_angle()
        elif rotation_choice == 1:  # Only Bob rotates
            alice_angle, bob_angle = KMC.alice.get_angle(), 0
        else:  # Both rotate
            alice_angle, bob_angle = 0, 0

        # Rotate motors and measure timing
        start_time = time.time()
        KMC.rotate_simulataneously(alice_angle, bob_angle, wait_for_elapsed_time=interval)
        elapsed = time.time() - start_time
        time_log.append(elapsed)

        # Adjust interval based on elapsed time
        interval += target_interval - elapsed

        # Next rotation to angles (25, 25)
        if rotation_choice == 0:  # Only Alice rotates
            alice_angle, bob_angle = 25, KMC.bob.get_angle()
        elif rotation_choice == 1:  # Only Bob rotates
            alice_angle, bob_angle = KMC.alice.get_angle(), 25
        else:  # Both rotate
            alice_angle, bob_angle = 25, 25

        # Rotate motors and measure timing
        start_time = time.time()
        KMC.rotate_simulataneously(alice_angle, bob_angle, wait_for_elapsed_time=interval)
        elapsed = time.time() - start_time
        time_log.append(elapsed)

        # Adjust interval again
        interval += target_interval - elapsed

    # Calculate and print statistics
    mean_time = statistics.mean(time_log)
    std_dev_time = statistics.stdev(time_log)

    print(f"Measured interval: {sum(time_log) / (2 * n):.6f}s")
    print(f"Mean interval: {mean_time:.6f}s")
    print(f"Standard deviation: {std_dev_time:.6f}s")
    print(f"Time log (s): {time_log}")

# Example usage
initial_interval = 0.536  # Start with the desired interval
target_interval = 0.536   # Desired interval
n = 20                    # Number of clicks

adaptive_rotation_efficient(KMC, initial_interval, target_interval, n)


Measured interval: 0.536056s
Mean interval: 0.536056s
Standard deviation: 0.007819s
Time log (s): [0.5508432388305664, 0.5227868556976318, 0.5442037582397461, 0.5318155288696289, 0.5424463748931885, 0.527700662612915, 0.5434112548828125, 0.5312845706939697, 0.5444145202636719, 0.5263833999633789, 0.5356705188751221, 0.5385072231292725, 0.5438973903656006, 0.5305685997009277, 0.529637336730957, 0.546926736831665, 0.5302023887634277, 0.5310933589935303, 0.5449833869934082, 0.5292785167694092, 0.5441296100616455, 0.5288023948669434, 0.5297548770904541, 0.5485959053039551, 0.5247809886932373, 0.5413298606872559, 0.5388469696044922, 0.5389301776885986, 0.5256664752960205, 0.5383126735687256, 0.5452492237091064, 0.5348091125488281, 0.5306746959686279, 0.530747652053833, 0.5438590049743652, 0.5315799713134766, 0.5448930263519287, 0.5275368690490723, 0.541872501373291, 0.5258030891418457]


In [10]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
TTC.displayCountTraces()

Output()

No such comm: 535cd2408d614ecea52f3ee10d7af6c1


In [11]:
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(3)
TTC.set_alice_transmission_channel(2)
# set bob channels
TTC.set_bob_reflection_channel(4)
TTC.set_bob_transmission_channel(1)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()



In [12]:
_ = TTC.performDelayAdjustment(integration_time=5)#, manual_delays=[0, 947, 78, -502])

KeyboardInterrupt: 

In [13]:
KMC.rotate_simulataneously(0, 0)

In [14]:
KMC.rotate_simulataneously(0, 45)

In [15]:
n_values = 150
trace_width = 3
TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

Output()

Output()

No such comm: 6afaa326024740c591cfa96d2fe0b64d
No such comm: d2a0f3b107a5406e933b93ce0ff2d389


In [16]:
KMC.rotate_simulataneously(0, 0, wait_for_completion=False)

In [17]:

KMC.rotate_simulataneously(0, 45, wait_for_completion=False)

In [10]:
# TODO
# coincidence channel handling and destruction
#TTC.coincidences_vchannels.getChannels()

In [19]:
# TODO This kills the coincidence channels in the visualisation. Needs to be worked on
idealAngles = [0, -22.5, 11.25, -11.25] # for triplett |HH> - |VV> initial state
johannes_ideal_angles = [0, 45/2, -22.5/2, 22.5/2]
johannes_classical_angles = [0, 45/2, -90/2, -45/2]

In [20]:
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=2, TTSimulator=None, coincidence_window_SI=0.5e-9, debug=True)



corr[a,b] = 1.0
	N[|T,T>]=     1	(1.000)
	N[|T,R>]=     0	(0.000)
	N[|R,T>]=     0	(0.000)
	N[|R,R>]=     0	(0.000)


\\wsl.localhost\Ubuntu\home\benjaminorthner\projects\bruQner\calibration_and_testing\..\src\time_tagger\time_tagger_controller.py:350: RuntimeWarning:

invalid value encountered in scalar divide

\\wsl.localhost\Ubuntu\home\benjaminorthner\projects\bruQner\calibration_and_testing\..\src\time_tagger\time_tagger_controller.py:354: RuntimeWarning:

invalid value encountered in scalar divide




corr[a,B] = nan
	N[|T,T>]=     0	(nan )
	N[|T,R>]=     0	(nan )
	N[|R,T>]=     0	(nan )
	N[|R,R>]=     0	(nan )

corr[A,b] = 1.0
	N[|T,T>]=     1	(1.000)
	N[|T,R>]=     0	(0.000)
	N[|R,T>]=     0	(0.000)
	N[|R,R>]=     0	(0.000)

corr[A,B] = 1.0
	N[|T,T>]=     1	(1.000)
	N[|T,R>]=     0	(0.000)
	N[|R,T>]=     0	(0.000)
	N[|R,R>]=     0	(0.000)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = nan


In [28]:
TTC.measure_S_with_two_ports(CHSH_angles=idealAngles, debug=True)

SPCM Pairs: ['TT', 'TR', 'RT', 'RR']

corr[a,b] = [ 0.691 -0.675 -0.586  0.601]
	N[a , b ]=[1026  237  209 1633]
	N[a , b ]=[ 189 1256  947  361]
	N[a , b ]=[  90  663 1054  516]
	N[a , b ]=[ 501  136  314 1883]

corr[a,B] = [ 0.664 -0.676 -0.699  0.695]
	N[a , B ]=[1002  221  189 1650]
	N[a , B ]=[ 167 1336  961  336]
	N[a , B ]=[ 127  678 1172  352]
	N[a , B ]=[ 456  169  189 2169]

corr[A,b] = [ 0.461 -0.415 -0.449  0.398]
	N[A , b ]=[ 744  195  376 1404]
	N[A , b ]=[157 968 866 656]
	N[A , b ]=[346 766 924 636]
	N[A , b ]=[ 621  522  304 1595]

corr[A,B] = [-0.561  0.546  0.486 -0.45 ]
	N[A , B ]=[ 277  752 1011  477]
	N[A , B ]=[ 570  370  290 1584]
	N[A , B ]=[ 839  141  365 1494]
	N[A , B ]=[119 990 885 690]

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = TT, TR, RT, RR : [2.3780884  2.31137612 2.22037178 2.14341162]


In [97]:
SimulatorPhiPlus = TT_Simulator(two_particle_states['phi_plus'], initial_state_noise=0.00, debug=False)
SimulatorPhiMinus = TT_Simulator(two_particle_states['phi_minus'], initial_state_noise=0.00, debug=False)
TTC.measureS(CHSH_angles=johannes_classical_angles, integration_time_per_basis_setting_SI=1, TTSimulator=SimulatorPhiMinus, coincidence_window_SI=0.5e-9)


corr[a,b] = -1.0
	N[|T,T>]=     0	(0.000)
	N[|T,R>]=  2539	(0.508)
	N[|R,T>]=  2461	(0.492)
	N[|R,R>]=     0	(0.000)

corr[a,B] = 0.0136
	N[|T,T>]=  1277	(0.255)
	N[|T,R>]=  1234	(0.247)
	N[|R,T>]=  1232	(0.246)
	N[|R,R>]=  1257	(0.251)

corr[A,b] = -0.0068
	N[|T,T>]=  1249	(0.250)
	N[|T,R>]=  1258	(0.252)
	N[|R,T>]=  1259	(0.252)
	N[|R,R>]=  1234	(0.247)

corr[A,B] = 1.0
	N[|T,T>]=  2485	(0.497)
	N[|T,R>]=     0	(0.000)
	N[|R,T>]=     0	(0.000)
	N[|R,R>]=  2515	(0.503)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 1.9932


# OSC Server Setup


In [17]:
my_ip = '192.168.0.2'
my_port = 7401 
clemens_port = 7401
visual_port = 7403

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=clemens_port, name='Clemens')
Visuals = OSCTarget(ip=my_ip, port=visual_port, name='Visuals')

# Start the server
OSC.start_server()

OSC Server Started. Serving on ('192.168.0.2', 7401)
----------------------------------------------------


State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: C_single
State Changed to: Q_diagonal
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_diagonal
State Changed to: Q_diagonal
State Changed to: C_diagonal
State Changed to: Q_diagonal
State Changed to: Q_all
State Changed to: C_single
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_all
State Changed to: Q_diagonal
State Changed to: Q_diagonal
State Changed to: Q_diagonal
State Changed to: C_diagonal
State Changed to: Q_diagonal
State Changed to: Q_diagonal
State Changed to: C_diagonal
State Changed to: Q_diagonal
State Changed to: C_diagonal
State Changed to: Q_diagonal
State Changed to: C_diagonal
State Changed to: Q_diagonal
State Changed to: C_d

In [11]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Clemens, measurement_results=measurement_results)


Measurement [1, 2, 3, 4] sent to Clemens


In [12]:
OSC.send_visuals(target=Visuals, measurement_results="hi")

Measurement hi sent to Visuals


In [26]:
# Global flag to control the loop
performance_running = False
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = [0, -22.5, 11.5, -11.5]
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, 11.5), (0, -11.5), (-22.5, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'Q_diagonal': [(0, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

def performance_cycle():
    """The function you want to execute repeatedly."""
    global performance_running
    
    time_of_last_visual = time.time()
    while performance_running:
        theta_a, theta_b = random.choice(angle_pair_dict[OSC.currently_selected_state])
        
        measurement_results = TTC.get_single_measurement(theta_a, theta_b)
        
        # check if setup is measuring Quantum or Classical angles for conversion to clemens format        
        if OSC.currently_selected_state[0] == 'Q':
            check_angles = Q_angles
        elif OSC.currently_selected_state[0] == 'C':
            check_angles = C_angles

        resultClemensFormat = [1 if theta_a == check_angles[0] else 2, 1 if theta_b == check_angles[2] else 2 ,*resultTranslate[measurement_results]]

        OSC.send_measurement(Clemens, resultClemensFormat)

        #if time.time() - time_of_last_visual > 7:
        #    OSC.send_visuals(Visuals, resultClemensFormat)
        #    time_of_last_visual = time.time()


def start_performance():
    """Starts the task in a separate thread."""
    global performance_running
    performance_running = True  # Ensure the loop is enabled
    task_thread = threading.Thread(target=performance_cycle, daemon=True)
    task_thread.start()

def stop_performance():
    """Signals the task to stop gracefully."""
    global performance_running
    performance_running = False
    print('performance stopped...')


In [32]:
Myself.client.send_message("/bruQner/set/state", 'Q_all')

In [25]:
stop_performance()

performance stopped...


In [ ]:
start_performance()

Measurement [2, 1, -1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [2, 2, -1, -1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 1, 1, -1] sent to Clemens
Measurement [2, 1, -1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [2, 1, -1, -1] sent to Clemens
Measurement [2, 1, -1, -1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [2, 2, 1, -1] sent to Clemens
Measurement [2, 2, 1, 1] sent to Clemens
Measurement [2, 2, -1, -1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [2, 2, -1, 1] sent